# Downloading from GEO datasets

This is meant to serve as proof of concept for searching, selecting, and downloading datasets (no profiles) from the GEO databank. Profiles can be added as a search option by changing the database name in the constructed address from "gds" to "geoprofiles". Search terms may need to be adjusted as well.

In [1]:
import GEOparse
import json # used to read JSON from eSearch/eSummary/eFetch requests
import urllib.request # used to read JSON from eSearch/eSummary/eFetch requests

The variables below serve as a reference of the field types that can be used to refine searches. The variables set to none are not used here.

In [2]:
# these phrases/search terms will be hardcoded (not editable)
URL_STEM_ESEARCH = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?"
URL_STEM_ESUM = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?"
DB_NAME = "db=gds" # geo database name
ID_TYPE = "&idtype=acc" # to retrieve accession data and not just ids (necessary to construct FTP address)
TERM = "&term=" # for search term concat
RET_MODE = "&retmode=json" # to get the information in JSON format

# these search terms will be input boxes (potential search terms)
# note: and/or/not boolean operators are allowed (and must be capitalized, or/not not so)
#       wildcards (*) are allowed
any_field = None # [All fields]
author = None # [Author] * is suppoorted, initials are optional
dataset_type = None #[DataSet Type] dropdown list populated with valid search terms
description = None # [DESC] * is supported
entry_type = None # [Entry Type] dropdown list populated with valid search terms
rec_filter = None # [Filter] dropdown list populated with valid search terms
geo_accession = None # [GEO Accession] valid dataset accession
mesh_terms = None # [MeSH Terms] Medical subject headings terms, * is supported
num_platform_probes = None # [Number of Platform Probes] integer, range is supported
num_samples = None # [Number of Samples] integer, range is supported
organism = None # [Organism] * is supported, must use NCBI taxonomy terms
platform_tech_type = None # [Platform Technology Type] dropdown list populated with valid search terms
project = None # [Project] dropdown list populated with valid search terms
pub_date = None # [Publication Date] format YYYY/MM range is supported
related_platform = None # [Related Platform] must be valid dataset accession
related_series = None # [Related Series] must be valid dataset accession
reporter_id = None # [Reporter Identifier] * is supported
sample_source = None # [Sample Source] * is supported
sample_type = None # [Sample Type] dropdown list populated with valid search terms
sample_val_type = None # [Sample Value Type] dropdown list populated with valid search terms
submitter_inst = None # [Institute] 
subset_desc = None # [Subset Description] * is supported
subset_var_type = None # [Subset Variable Type] dropdown list populated with valid search terms
supp_files = None # [Supplementary Files] * is supported
tag_len = None # [Tag Length] integer
title = None # [Title] * is supported
update_date = None # [Update Date] format YYYY/MM range is supported

## Searching

The fields above can be used to further customize searches. Examples of how to construct eSearches using these terms can be found [here](https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch).

Now the eSearch search must be constructed

In [3]:
search_term = "diet AND human AND coriobacterineae[Organism] AND attribute name strain[Filter]"

The search term will be formatted as:

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&term=diet+AND+human+AND+coriobacterineae[Organism]+AND+attribute+name+strain[Filter]

In the widget, these will be selected somehow (dropdown menu as with GEO search, or as a set of input boxes depending on the dynamicism of bwb). These will be concatenated until they are in the form of the variable "search_term" above. This is when the and/or/not will be capitalized and the search term will be appended.

In [4]:
def buildSearchTerm(search_input):
    # build with hardcoded stem
    url_string = URL_STEM_ESEARCH + DB_NAME + ID_TYPE + TERM
    # split string for iterating
    input_list = search_input.split(" ")
    # for each word in search term
        # add "+" between each word
    for word in input_list:
        url_string += word + "+"
    url_string = url_string[:-1] # remove trailing "+"
    url_string += RET_MODE # append JSON request
    return url_string

In [5]:
search_term_formatted = buildSearchTerm(search_term)

### Get eSearch results

This search term is used to get UIDs for eSummary display information. The JSON printed below contains the field needed next; idlist

In [6]:
# download JSON (with help from https://stackoverflow.com/questions/12965203/how-to-get-json-from-webpage-into-python-script)
with urllib.request.urlopen(search_term_formatted) as url:
    data = json.loads(url.read().decode())
# printed nicer with the help of https://stackoverflow.com/questions/12943819/how-to-prettyprint-a-json-file
print(json.dumps(json.loads(json.dumps(data)), indent=4, sort_keys=True))

{
    "esearchresult": {
        "count": "4",
        "idlist": [
            "200048537",
            "200048809",
            "200025576",
            "200025575"
        ],
        "querytranslation": "(\"diet\"[MeSH Terms] OR diet[All Fields]) AND (\"humans\"[MeSH Terms] OR \"Homo sapiens\"[Organism] OR human[All Fields]) AND \"Coriobacteriales\"[Organism] AND attribute name strain[Filter]",
        "retmax": "4",
        "retstart": "0",
        "translationset": [
            {
                "from": "diet",
                "to": "\"diet\"[MeSH Terms] OR diet[All Fields]"
            },
            {
                "from": "human",
                "to": "\"humans\"[MeSH Terms] OR \"Homo sapiens\"[Organism] OR human[All Fields]"
            },
            {
                "from": "coriobacterineae[Organism]",
                "to": "\"Coriobacteriales\"[Organism]"
            }
        ],
        "translationstack": [
            {
                "count": "120051",
           

### Get eSummary results

Save the id list as a variable, and use these values to fetch summaries

In [7]:
id_list = data["esearchresult"]["idlist"]
# create empty list to store JSONs for each selected id
list_of_sum_jsons = []

In [8]:
for id in id_list:
    esum_url_string = URL_STEM_ESUM + DB_NAME + ID_TYPE + "&id=" + id + RET_MODE
    print(esum_url_string)
    with urllib.request.urlopen(esum_url_string) as url:
        data = json.loads(url.read().decode())
    list_of_sum_jsons.append(data.copy())

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&idtype=acc&id=200048537&retmode=json
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&idtype=acc&id=200048809&retmode=json
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&idtype=acc&id=200025576&retmode=json
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&idtype=acc&id=200025575&retmode=json


In [9]:
# printed nicer with the help of https://stackoverflow.com/questions/12943819/how-to-prettyprint-a-json-file
print(json.dumps(json.loads(json.dumps(data)), indent=4, sort_keys=True))

{
    "header": {
        "type": "esummary",
        "version": "0.3"
    },
    "result": {
        "200025575": {
            "accession": "GSE25575",
            "bioproject": "PRJNA142411",
            "entrytype": "GSE",
            "extrelations": [],
            "ftplink": "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE25nnn/GSE25575/",
            "gds": "",
            "gdstype": "Expression profiling by array",
            "geo2r": "yes",
            "gpl": "9803",
            "gse": "25575",
            "n_samples": 25,
            "pdat": "2011/05/01",
            "platformtaxa": "",
            "platformtitle": "",
            "projects": [],
            "ptechtype": "",
            "pubmedids": [],
            "relations": [],
            "samples": [
                {
                    "accession": "GSM628392",
                    "title": "oat spelt xylan_2"
                },
                {
                    "accession": "GSM628409",
                    "title": "in

### Construct FTP to download datasets

In [10]:
tostr = "" + list_of_sum_jsons[0]["result"]["uids"][0]
print(list_of_sum_jsons[0]["result"][tostr]["accession"])

GSE48537


From this details can be displayed to a user so they can select all datasets they want to download

Now grab the accession numbers to be fed to GEOparse

In [11]:
# empty list to store accession numbers
acc_nums = []

In [12]:
uid_count = 0
for result in list_of_sum_jsons:
    uid = "" + id_list[uid_count] # grab uid for json lookup, store as str
    uid_count += 1
    acc_nums.append(result["result"][uid]["accession"])

In [13]:
print(acc_nums)

['GSE48537', 'GSE48809', 'GSE25576', 'GSE25575']


### Download selected datasets

Now use the accession numbers to download datasets

In [14]:
for acc_num in acc_nums:
    ds = GEOparse.get_GEO(geo=acc_num, destdir="./")

30-Aug-2019 10:26:40 DEBUG utils - Directory ./ already exists. Skipping.
30-Aug-2019 10:26:40 INFO GEOparse - File already exist: using local version.
30-Aug-2019 10:26:40 INFO GEOparse - Parsing ./GSE48537_family.soft.gz: 
30-Aug-2019 10:26:40 DEBUG GEOparse - DATABASE: GeoMiame
30-Aug-2019 10:26:40 DEBUG GEOparse - SERIES: GSE48537
30-Aug-2019 10:26:40 DEBUG GEOparse - PLATFORM: GPL9803
30-Aug-2019 10:26:40 DEBUG GEOparse - PLATFORM: GPL11535
30-Aug-2019 10:26:40 DEBUG GEOparse - PLATFORM: GPL17329
30-Aug-2019 10:26:40 DEBUG GEOparse - PLATFORM: GPL17330
30-Aug-2019 10:26:40 DEBUG GEOparse - PLATFORM: GPL17468
30-Aug-2019 10:26:40 DEBUG GEOparse - SAMPLE: GSM1171968
30-Aug-2019 10:26:40 DEBUG GEOparse - SAMPLE: GSM1171969
30-Aug-2019 10:26:40 DEBUG GEOparse - SAMPLE: GSM1171970
30-Aug-2019 10:26:40 DEBUG GEOparse - SAMPLE: GSM1171971
30-Aug-2019 10:26:40 DEBUG GEOparse - SAMPLE: GSM1171972
30-Aug-2019 10:26:40 DEBUG GEOparse - SAMPLE: GSM1171973
30-Aug-2019 10:26:40 DEBUG GEOparse -

30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172101
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172102
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172103
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172104
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172105
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172106
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172107
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172108
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172109
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172110
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172111
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172112
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172113
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172114
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172115
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172116
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172117
30-Aug-2019 10:26:41 DEBUG GEOp

30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172245
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172246
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172247
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172248
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172249
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172250
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172251
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172252
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172253
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172254
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172255
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172256
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172257
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172258
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172259
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172260
30-Aug-2019 10:26:41 DEBUG GEOparse - SAMPLE: GSM1172261
30-Aug-2019 10:26:41 DEBUG GEOp

30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172389
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172390
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172391
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172392
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172393
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172394
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172395
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172396
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172397
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172398
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172399
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172400
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172401
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172402
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172403
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172404
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172405
30-Aug-2019 10:26:42 DEBUG GEOp

30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172533
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172534
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172535
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172536
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172537
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172538
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172539
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172540
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172541
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172542
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172543
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172544
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172545
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172546
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172547
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172548
30-Aug-2019 10:26:42 DEBUG GEOparse - SAMPLE: GSM1172549
30-Aug-2019 10:26:42 DEBUG GEOp

30-Aug-2019 10:26:55 DEBUG GEOparse - SAMPLE: GSM1180447
30-Aug-2019 10:26:55 DEBUG GEOparse - SAMPLE: GSM1180448
30-Aug-2019 10:26:55 DEBUG GEOparse - SAMPLE: GSM1180449
30-Aug-2019 10:26:55 DEBUG GEOparse - SAMPLE: GSM1180450
30-Aug-2019 10:26:55 DEBUG GEOparse - SAMPLE: GSM1180451
30-Aug-2019 10:26:56 DEBUG GEOparse - SAMPLE: GSM1180452
30-Aug-2019 10:26:56 DEBUG GEOparse - SAMPLE: GSM1180453
30-Aug-2019 10:26:56 DEBUG GEOparse - SAMPLE: GSM1180454
30-Aug-2019 10:26:56 DEBUG GEOparse - SAMPLE: GSM1180455
30-Aug-2019 10:26:56 DEBUG GEOparse - SAMPLE: GSM1180456
30-Aug-2019 10:26:56 DEBUG GEOparse - SAMPLE: GSM1180457
30-Aug-2019 10:26:56 DEBUG GEOparse - SAMPLE: GSM1180458
30-Aug-2019 10:26:56 DEBUG GEOparse - SAMPLE: GSM1180459
30-Aug-2019 10:26:57 DEBUG GEOparse - SAMPLE: GSM1180460
30-Aug-2019 10:26:57 DEBUG GEOparse - SAMPLE: GSM1180461
30-Aug-2019 10:26:57 DEBUG GEOparse - SAMPLE: GSM1180462
30-Aug-2019 10:26:57 DEBUG GEOparse - SAMPLE: GSM1180463
30-Aug-2019 10:26:57 DEBUG GEOp

30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191844
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191845
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191846
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191847
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191848
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191849
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191850
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191851
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191852
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191853
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191854
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191855
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191856
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191857
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191858
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191859
30-Aug-2019 10:27:04 DEBUG GEOparse - SAMPLE: GSM1191860
30-Aug-2019 10:27:04 DEBUG GEOp

30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185038
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185039
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185040
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185041
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185042
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185043
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185044
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185045
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185046
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185047
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185048
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185049
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185050
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185051
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185052
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185053
30-Aug-2019 10:27:06 DEBUG GEOparse - SAMPLE: GSM1185054
30-Aug-2019 10:27:06 DEBUG GEOp

30-Aug-2019 10:31:55 DEBUG GEOparse - SAMPLE: GSM628409
30-Aug-2019 10:31:55 DEBUG GEOparse - SAMPLE: GSM628410
30-Aug-2019 10:31:55 DEBUG GEOparse - SAMPLE: GSM628411
30-Aug-2019 10:31:56 DEBUG GEOparse - SAMPLE: GSM628412
30-Aug-2019 10:31:56 DEBUG GEOparse - SAMPLE: GSM628413


## Test downloads

In [15]:
# example code from NCBI GEO FAQs
print("GSM example:")
for gsm_name, gsm in ds.gsms.items():
    print("Name: ", gsm_name)
    print("Metadata:",)
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print ("Table data:",)
    print (gsm.table.head())
    break

GSM example:
Name:  GSM628389
Metadata:
 - title : glucose_1
 - geo_accession : GSM628389
 - status : Public on May 01 2011
 - submission_date : Nov 23 2010
 - last_update_date : May 01 2011
 - type : RNA
 - channel_count : 1
 - source_name_ch1 : in vitro bacterial culture, glucose
 - organism_ch1 : Bacteroides ovatus
 - taxid_ch1 : 28116
 - characteristics_ch1 : strain: ATCC 8483, treatment: glucose, isolation source: in vitro culture
 - treatment_protocol_ch1 : In vitro-grown bacteria or mouse cecal contents were treated with RNAProtect reagent (Qiagen) and frozen at -80C until RNA extraction.
 - growth_protocol_ch1 : Bacteria were grown either in vitro to mid-exponential phase on the defined carbon source indicated; or, extracted from the ceca of gnotobiotic mice colonized with only B. ovatus.
 - molecule_ch1 : total RNA
 - extract_protocol_ch1 : RNA was extracted directly from in vitro-grown bacteria. In vivo-grown cells were subjected to an initial phenol/chloroform bead-beating s

In [16]:
print("GPL example:")
for gpl_name, gpl in ds.gpls.items():
    print("Name: ", gpl_name)
    print("Metadata:",)
    for key, value in gpl.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print("Table data:",)
    print(gpl.table.head())
    break

GPL example:
Name:  GPL9803
Metadata:
 - title : [SynComm1a520717] SynComm1: Synthetic Human Gut Community GeneChip, v1.0
 - geo_accession : GPL9803
 - status : Public on Dec 18 2009
 - submission_date : Dec 15 2009
 - last_update_date : Jul 03 2013
 - technology : in situ oligonucleotide
 - distribution : custom-commercial
 - organism : Bacteroides thetaiotaomicron VPI-5482, Bacteroides sp. WH2, Blautia obeum ATCC 29174, [Ruminococcus] torques ATCC 27756, Dorea longicatena DSM 13814, [Clostridium] scindens ATCC 35704, Bacteroides ovatus ATCC 8483, Bacteroides uniformis ATCC 8492, Faecalibacterium prausnitzii M21/2, Bacteroides caccae ATCC 43185, Collinsella aerofaciens ATCC 25986, [Clostridium] spiroforme DSM 1552, Bacteroides vulgatus ATCC 8482, Parabacteroides distasonis ATCC 8503, [Eubacterium] rectale ATCC 33656
 - taxid : 226186, 311784, 411459, 411460, 411462, 411468, 411476, 411479, 411485, 411901, 411903, 428126, 435590, 435591, 515619
 - manufacturer : Affymetrix
 - manufactu